In [ ]:
now - datetime.timedelta(minutes=4)

In [ ]:
then = now - datetime.timedelta(hours=6)

In [1]:
from flask import render_template, request
from app import app
from models import Historical, Forecast, Station, Current
import datetime

import plotly
import plotly.graph_objs as go 

import pandas as pd
import numpy as np 
import json

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1='#1f77b4' #blue
c2='green' #green
n=500

fig, ax = plt.subplots(figsize=(8, 5))
for x in range(n+1):
    ax.axvline(x, color=colorFader(c1,c2,x/n), linewidth=4) 
plt.show()



In [4]:
now = datetime.datetime.now().replace(microsecond=0,second=0,minute=0)
for n in range (1,24):
    print(now - datetime.timedelta(hours=n))

2019-06-21 21:00:00
2019-06-21 20:00:00
2019-06-21 19:00:00
2019-06-21 18:00:00
2019-06-21 17:00:00
2019-06-21 16:00:00
2019-06-21 15:00:00
2019-06-21 14:00:00
2019-06-21 13:00:00
2019-06-21 12:00:00
2019-06-21 11:00:00
2019-06-21 10:00:00
2019-06-21 09:00:00
2019-06-21 08:00:00
2019-06-21 07:00:00
2019-06-21 06:00:00
2019-06-21 05:00:00
2019-06-21 04:00:00
2019-06-21 03:00:00
2019-06-21 02:00:00
2019-06-21 01:00:00
2019-06-21 00:00:00
2019-06-20 23:00:00


In [43]:
def query_over_period(m):
    d = {}
    for n in range (1,48):

        [print(x.drybulb, x.id, x.retrieval_time) for x in Forecast.query.filter(
        Forecast.id == now - datetime.timedelta(hours=n),
        Forecast.retrieval_time == Forecast.id - datetime.timedelta(hours=m)
        )]
        
        
        

In [49]:
query_over_period(24)

19 2019-06-21 18:00:00 2019-06-20 18:00:00


In [16]:
Forecast.retrieval_time == now - datetime.timedelta(hours=m),

datetime.datetime(2019, 6, 21, 18, 0)

In [ ]:
data

In [ ]:
data = []
def hourly_plot():

    for n in range(24):
        x = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

        y = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=n),
            Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]   


        forecasts = go.Scatter(
                x=x,
                y=y,
                name='forecasts_{}h'.format(n),
                line=dict(
                    color = ('rgb(0,76,153)'),
                    width = 2,)
                )
        data.append(forecasts)


In [ ]:
data = [historicals, actuals, forecasts_latest, forecasts_6h, forecasts_1d, forecasts_2d]
fig = go.Figure(data=data, layout=layout)
graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
return graphJSON



In [ ]:
    # 6h   
    x4 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=4),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

    y4 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=4),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]


In [ ]:
@app.route('/')
def index():   

    plot = create_plot()
    return render_template("index.html", plot=plot)    
    
def create_plot():
    N = 40
    now = datetime.datetime.now()
    x00 = [x.id for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]
    y00 = [x.drybulb for x in Historical.query.filter(Historical.id >= now - datetime.timedelta(days=7)).order_by(Historical.id)]

    # actuals
    x0 = [x.id for x in Current.query.filter(Current.id >= now - datetime.timedelta(days=7)).order_by(Current.id)]
    y0 = [x.drybulb for x in Current.query.filter(Current.id >= now - datetime.timedelta(days=7)).order_by(Current.id)]

    # latest forecast (retrieved within the last hour)   
    x1 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=1))
          .order_by(Forecast.id)]

    y1 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=1))
          .order_by(Forecast.id)]

    # 6h   
    x4 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=6),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]

    y4 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time >= now - datetime.timedelta(hours=6),
    	Forecast.retrieval_time < now - datetime.timedelta(minutes=355)).order_by(Forecast.id)]
        
    # retrieved on roughly this hour, 1d ago
    # datetime greater than now minus 1d; less than five minutes after now minus 1d   
    x2 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    y2 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=1),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=1435)).order_by(Forecast.id)]

    # retrieved on roughly this hour, 2d ago
    # datetime greater than now minus 2d; less than five minutes after now minus 2d   
    x3 = [x.id for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    y3 = [x.drybulb for x in Forecast.query.filter(Forecast.retrieval_time > now - datetime.timedelta(days=2),
        Forecast.retrieval_time < now - datetime.timedelta(minutes=2875)).order_by(Forecast.id)]

    historicals = go.Scatter(
            x=x00,
            y=y00,
            name='historicals',
            line=dict(
                color = ('rgb(192,192,192)'),
                width = 2,)
            )

    actuals = go.Scatter(
            x=x0,
            y=y0,
            name='actuals',
            mode='lines',
            line=dict(
                color = ('rgb(255,0,127)'),
                width = 3,)
            )
    
    forecasts_latest = go.Scatter(
            x=x1,
            y=y1,
            name='forecasts_latest',
            line=dict(
                color = ('rgb(0,51,102)'),
                width = 2,)
            )

    forecasts_1d = go.Scatter(
            x=x2,
            y=y2,
            name='forecasts_1d',
            line=dict(
                color = ('rgb(0,76,153)'),
                width = 2,)
            )

    forecasts_2d = go.Scatter(
            x=x3,
            y=y3,
            name='forecasts_2d',
            line=dict(
                color = ('rgb(0,102,204)'),
                width = 2,)
            )

    forecasts_6h = go.Scatter(
            x=x4,
            y=y4,
            name='forecasts_6h',
            line=dict(
                color = ('rgb(0,128,255)'),
                width = 2,)
            )

    layout = go.Layout(
    title=go.layout.Title(
        text='Weather Forecast Tracker',
            font=dict(
                family='Helvetica, monospace',
                size=20
	            )
    ),

    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Week of {}'.format((now - datetime.timedelta(days=now.weekday())).date()),
            font=dict(
                family='Helvetica, monospace',
                size=18
            )
        )
    ),

    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Degrees C',
            font=dict(
                family='Helvetica, monospace',
                size=18
	            )
	        )
	    )
	)

    

    data = [historicals, actuals, forecasts_latest, forecasts_6h, forecasts_1d, forecasts_2d]
    fig = go.Figure(data=data, layout=layout)
    graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return graphJSON


In [ ]:
create_plot()